In [20]:
import os
import cv2
import numpy as np
from datetime import datetime, timedelta
from glob import glob

In [35]:
def get_file_paths(folder_path, extensions=None):
    """
    Obtiene todas las rutas de archivos en una carpeta que coincidan con las extensiones especificadas.

    Args:
        folder_path (str): Ruta de la carpeta a explorar.
        extensions (list[str]): Lista de extensiones de archivo a incluir (e.g., ['.jpg', '.png']).

    Returns:
        list[str]: Lista de rutas completas de los archivos.
    """
    if extensions is None:
        extensions = [".jpg"]  # Extensión predeterminada

    file_paths = [
        os.path.join(folder_path, f)
        for f in os.listdir(folder_path)
        if any(f.endswith(ext) for ext in extensions)
    ]
    return file_paths

def extract_timestamp(file_path):
    """
    Extrae el timestamp del nombre del archivo.

    Args:
        file_path (str): Ruta completa del archivo.

    Returns:
        datetime: Timestamp extraído del nombre del archivo.
    """
    try:
        file_name = os.path.basename(file_path)
        timestamp_str = file_name.split("_")[3][1:15]  # 'sYYYYMMDDHHMMSS'
        return datetime.strptime(timestamp_str, "%Y%m%d%H%M%S")
    except Exception as e:
        raise ValueError(f"Error al extraer timestamp del archivo {file_path}: {e}")
    
def load_images_in_range(folder_path, start_date, end_date):
    """
    Carga imágenes en un rango de fechas especificado.

    Args:
        folder_path (str): Ruta de la carpeta que contiene las imágenes.
        start_date (datetime): Fecha inicial del rango.
        end_date (datetime): Fecha final del rango.

    Returns:
        list[np.array], list[datetime]: Lista de imágenes y sus timestamps correspondientes.
    """
    files = glob(os.path.join(folder_path, "*.jpg"))
    images = []
    timestamps = []

    for file in files:
        try:
            timestamp = extract_timestamp(file)
            if start_date <= timestamp <= end_date:
                img = cv2.imread(file)
                if img is not None:
                    images.append(img)
                    timestamps.append(timestamp)
        except Exception as e:
            print(f"Error procesando archivo {file}: {e}")

    # Ordenar por timestamp
    sorted_data = sorted(zip(timestamps, images))
    if sorted_data:
        timestamps, images = zip(*sorted_data)
        return list(images), list(timestamps)
    return [], []

# Preprocesar las imágenes
def preprocess_images(images, target_size):
    """
    Redimensiona y normaliza las imágenes.

    Args:
        images (list[np.array]): Lista de imágenes originales.
        target_size (tuple): Dimensiones a las que redimensionar las imágenes.

    Returns:
        np.array: Imágenes preprocesadas.
    """
    processed_images = []
    for img in images:
        resized_img = cv2.resize(img, target_size)  # Redimensionar
        normalized_img = resized_img / 255.0       # Normalizar a [0, 1]
        processed_images.append(normalized_img)
    return np.array(processed_images)

# Crear ventanas deslizantes
def create_sliding_windows(images, sequence_length, output_length):
    """
    Crea pares de entrada y salida usando ventanas deslizantes.

    Args:
        images (np.array): Imágenes preprocesadas.
        sequence_length (int): Longitud de la secuencia de entrada.
        output_length (int): Longitud de la secuencia de salida.

    Returns:
        np.array, np.array: Pares de entrada (X) y salida (y).
    """
    X, y = [], []
    for i in range(len(images) - sequence_length - output_length + 1):
        X.append(images[i:i + sequence_length])
        y.append(images[i + sequence_length:i + sequence_length + output_length])
    return np.array(X), np.array(y)

def find_gaps(timestamps, interval_minutes=10, tolerance_seconds=10):
    """
    Identifica gaps en los timestamps según un intervalo esperado.

    Args:
        timestamps (list[datetime]): Lista ordenada de timestamps.
        interval_minutes (int): Intervalo esperado entre timestamps (en minutos).

    Returns:
        list[tuple]: Lista de tuples que contienen (timestamp_anterior, timestamp_actual, delta).
    """
    gaps = []
    for i in range(1, len(timestamps)):
        delta = timestamps[i] - timestamps[i - 1]
        if delta > timedelta(minutes=interval_minutes, seconds=tolerance_seconds):  # Sin tolerancia
            gaps.append((timestamps[i - 1], timestamps[i], delta))
    return gaps

In [41]:
def main():
    """
    Función principal para el flujo completo del proyecto:
    - Preprocesamiento de datos
    - Configuración y entrenamiento del modelo
    - Predicción de imágenes faltantes
    """
    # Parámetros de configuración
    folder_path = "../data/Images"  # Carpeta con las imágenes
    extensions = [".jpg"]  # Extensiones de archivo válidas
    interval_minutes = 10  # Intervalo esperado entre imágenes
    target_size = (64, 64)  # Tamaño deseado de las imágenes
    sequence_length = 10  # Longitud de la secuencia de entrada
    output_length = 5  # Número de imágenes que el modelo debe predecir
    start_date = datetime(2023, 7, 1, 5, 0, 0)  # Fecha de inicio
    end_date = datetime(2023, 7, 8, 18, 30, 0)  # Fecha de fin

    # Paso 1: Obtener rutas de los archivos e identificar gaps
    file_paths = get_file_paths(folder_path, extensions)
    timestamps = sorted([extract_timestamp(fp) for fp in file_paths])
    gaps = find_gaps(timestamps, interval_minutes)
    print(f"Se encontraron {len(gaps)} gaps.")
    for gap in gaps:
        print(f"Gap entre {gap[0]} y {gap[1]}: {gap[2]}")

    # Paso 2: Cargar imágenes en el rango de fechas especificado
    images, timestamps_in_range = load_images_in_range(folder_path, start_date, end_date)
    print(f"Se cargaron {len(images)} imágenes en el rango de fechas.")
    if images:
        print(f"Primera imagen cargada: {timestamps_in_range[0]}")
        print(f"Última imagen cargada: {timestamps_in_range[-1]}")

    # Paso 3: Preprocesar las imágenes (redimensionar y normalizar)
    preprocessed_images = preprocess_images(images, target_size)

    # Paso 4: Crear ventanas deslizantes para las secuencias de entrenamiento
    X, y = create_sliding_windows(preprocessed_images, sequence_length, output_length)
    print(f"Secuencias de entrada creadas: {X.shape}")
    print(f"Secuencias de salida creadas: {y.shape}")

    # Paso 5: Configurar el modelo basado en VGG16 + LSTM
    input_shape = (target_size[0], target_size[1], 3)  # Altura, anchura, canales
    model = build_model(input_shape, sequence_length, output_length)

    # Visualización del modelo (opcional)
    plot_model(model, to_file="model_architecture.png", show_shapes=True)
    print(model.summary())

    # Paso 6: Entrenar el modelo
    history = model.fit(X, y, epochs=10, batch_size=16, validation_split=0.2)
    print("Entrenamiento completado.")

    # Paso 7: Predicción
    # Aquí puedes generar imágenes faltantes basadas en una secuencia de entrada proporcionada
    # Por ejemplo, tomar las últimas `sequence_length` imágenes para predecir
    test_input = X[-1].reshape(1, sequence_length, *input_shape)  # Usar la última secuencia de entrenamiento como ejemplo
    predicted_output = model.predict(test_input)
    print(f"Predicción completada. Forma de la salida: {predicted_output.shape}")

    # Paso 8: Postprocesamiento
    # Reconstruir las imágenes predichas desde el formato aplanado a (64, 64, 3)
    reconstructed_images = predicted_output.reshape(output_length, target_size[0], target_size[1], 3)
    print(f"Imágenes reconstruidas: {reconstructed_images.shape}")

    # Guardar las imágenes predichas (opcional)
    output_folder = "../data/Predicted_Images"
    os.makedirs(output_folder, exist_ok=True)
    for idx, img in enumerate(reconstructed_images):
        img_path = os.path.join(output_folder, f"predicted_{idx + 1}.jpg")
        cv2.imwrite(img_path, (img * 255).astype(np.uint8))  # Desnormalizar y guardar
        print(f"Imagen guardada: {img_path}")

if __name__ == "__main__":
    main()

Se encontraron 1 gaps.
Gap entre 2023-07-07 18:20:21 y 2023-07-10 05:00:20: 2 days, 10:39:59
Se cargaron 945 imágenes en el rango de fechas.
Primera imagen cargada: 2023-07-01 05:00:21
Última imagen cargada: 2023-07-07 18:20:21
Secuencias de entrada creadas: (931, 10, 64, 64, 3)
Secuencias de salida creadas: (931, 5, 64, 64, 3)
Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 10, 64, 64, 3)]   0         
                                                                 
 time_distributed_8 (TimeDi  (None, 10, 2048)          14714688  
 stributed)                                                      
                                                                 
 lstm_14 (LSTM)              (None, 10, 256)           2360320   
                                                                 
 lstm_15 (LSTM)              (None, 128)               1971

ValueError: in user code:

    File "/Users/santiagoromero/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Users/santiagoromero/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/santiagoromero/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Users/santiagoromero/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/santiagoromero/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/Users/santiagoromero/anaconda3/lib/python3.11/site-pac

# Configuracion del modelo

In [40]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, Flatten

def build_model(input_shape, sequence_length, output_length):
    """
    Construye el modelo basado en VGG16 + LSTM.

    El modelo utiliza VGG16 como extractor de características de las imágenes,
    seguido por una LSTM para modelar las relaciones temporales en la secuencia.
    Finalmente, una capa densa produce la salida, que corresponde a las imágenes generadas.

    Args:
        input_shape (tuple): Dimensiones de las imágenes de entrada (e.g., (64, 64, 3)).
        sequence_length (int): Longitud de la secuencia de entrada.
        output_length (int): Longitud de la secuencia de salida.

    Returns:
        Model: Modelo compilado listo para entrenar.
    """
    # Cargar la arquitectura de VGG16 preentrenada en ImageNet, excluyendo las capas superiores (clasificación)
    vgg16 = VGG16(weights="imagenet", include_top=False, input_shape=input_shape)
    
    # Congelar las capas de VGG16 para evitar que sus pesos se actualicen durante el entrenamiento
    for layer in vgg16.layers:
        layer.trainable = False

    # Construir un extractor de características con VGG16 seguido de un Flatten
    vgg16_extractor = Sequential([
        vgg16,  # Extraer características convolucionales
        Flatten()  # Convertir el volumen 3D a un vector 1D
    ])

    # Capa de entrada: secuencia de imágenes con dimensiones específicas
    input_layer = Input(shape=(sequence_length, *input_shape))  # (sequence_length, height, width, channels)

    # Extraer características de cada imagen en la secuencia usando TimeDistributed
    features = TimeDistributed(vgg16_extractor)(input_layer)  # Salida: (sequence_length, features_dim)

    # Modelar relaciones temporales con dos capas LSTM
    lstm_output = LSTM(256, return_sequences=True)(features)  # Primera LSTM, salida con secuencias completas
    lstm_output = LSTM(128, return_sequences=False)(lstm_output)  # Segunda LSTM, salida final como vector

    # Capa densa para producir la salida deseada: las próximas imágenes de la secuencia
    output_units = output_length * input_shape[0] * input_shape[1] * input_shape[2]  # Salida flattenizada
    output_layer = Dense(output_units, activation="sigmoid")(lstm_output)  # Activación sigmoide para normalización

    # Crear y compilar el modelo
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer="adam", loss="mse", metrics=["mae"])  # Compilar con optimizador Adam y pérdida MSE

    return model

# Parámetros del modelo
input_shape = (64, 64, 3)  # Dimensiones de las imágenes
sequence_length = 10  # Secuencia de entrada (ejemplo)
output_length = 5  # Secuencia de salida (predicción)

# Crear el modelo
model = build_model(input_shape, sequence_length, output_length)

# Resumen del modelo
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 10, 64, 64, 3)]   0         
                                                                 
 time_distributed_7 (TimeDi  (None, 10, 2048)          14714688  
 stributed)                                                      
                                                                 
 lstm_12 (LSTM)              (None, 10, 256)           2360320   
                                                                 
 lstm_13 (LSTM)              (None, 128)               197120    
                                                                 
 dense_9 (Dense)             (None, 61440)             7925760   
                                                                 
Total params: 25197888 (96.12 MB)
Trainable params: 10483200 (39.99 MB)
Non-trainable params: 14714688 (56.13 MB)
___________

Detalles Clave:

1.	VGG16:
- Se carga el modelo preentrenado en ImageNet.
- Se eliminan las capas de clasificación finales para usarlo como extractor de características.
- Se congela el entrenamiento de las capas preentrenadas.
2.	LSTM:
- Recibe secuencias de características extraídas por la VGG16.
- Tiene una capa Dense final para predecir las características de la siguiente imagen.
3.	Dimensiones:
- Asegúrate de que las imágenes de entrada tengan tamaño 64x64 (preprocesamiento).
- Define cuántos pasos temporales quieres para la secuencia (time_steps).

# Entrenamiento del modelo

In [33]:
def create_sliding_windows(images, sequence_length, output_length):
    """
    Crea pares de entrada y salida usando ventanas deslizantes.

    Args:
        images (np.array): Imágenes preprocesadas.
        sequence_length (int): Longitud de la secuencia de entrada.
        output_length (int): Longitud de la secuencia de salida.

    Returns:
        np.array, np.array: Pares de entrada (X) y salida (y).
    """
    X, y = [], []
    for i in range(len(images) - sequence_length - output_length + 1):
        X.append(images[i:i + sequence_length])
        y.append(images[i + sequence_length:i + sequence_length + output_length])
    return np.array(X), np.array(y)

# Crear datos de entrenamiento
X, y = create_sliding_windows(preprocessed_images, sequence_length, output_length)

# Entrenar el modelo
history = model.fit(X, y, epochs=10, batch_size=16, validation_split=0.2)

NameError: name 'preprocessed_images' is not defined

# Prediccion del modelo

In [ ]:
# Usar las últimas 10 imágenes para predecir las próximas 5
input_sequence = preprocessed_images[-sequence_length:]
input_sequence = np.expand_dims(input_sequence, axis=0)  # Expandir dimensiones para el modelo

predicted_images = model.predict(input_sequence)

# Postprocesamiento: convertir imágenes de [0, 1] a [0, 255]
predicted_images = (predicted_images * 255).astype(np.uint8)

NameError: name 'images' is not defined

In [42]:
import cv2

# Cargar la imagen para verificar sus dimensiones
image_path = "../data/Images/OR_ABI-L2-ACMF-M6_G16_s20230710141020.jpg"  # Cambia esto por la ruta de la imagen real
real_image = cv2.imread(image_path)
real_image_shape = real_image.shape  # Altura, Ancho, Canales
print(f"Dimensiones de la imagen: {real_image_shape}")


Dimensiones de la imagen: (449, 593, 3)
